<center><img src="images/2024_reInvent_Logo_wDate_Black_V3.png" alt="drawing" width="400" style="background-color:white; padding:1em;" /></center> <br/>

# <a name="0">re:Invent 2024 | Lab 1: Build your RAG powered chatbot  </a>
## <a name="0">Build a chatbot with Knowledge Bases and Guardrails to detect and remediate hallucinations </a>

## Lab Overview
In this lab, you will:
1. Take a deeper look at which LLM parameters influence or control for model hallucinations
2. Set up Retrieval Augmented Generation and understand how it can control for hallucinations
3. Apply contextual grounding in Amazon Bedrock Guardrails to intervene when a model hallucinates
4. Use RAGAS evaluation and understand which metrics help us measure hallucinations

## Dataset
For this workshop, we will use the [Bedrock User Guide](https://docs.aws.amazon.com/pdfs/bedrock/latest/userguide/bedrock-ug.pdf) available as a PDF file.
## Use-Case Overview
In this lab, we want to develop a chatbot which can answer questions about Amazon Bedrock as factually as possible. We will set up Retrieval Augmented Generation using [Amazon Bedrock Knowledge Bases](https://aws.amazon.com/bedrock/knowledge-bases/) and apply [Amazon Guardrails](https://aws.amazon.com/bedrock/guardrails/) to intervene when hallucinations are detected.


#### Lab Sections

This lab notebook has the following sections:
    
Please work top to bottom of this notebook and don't skip sections as this could lead to error messages due to missing code.


----

# Star Github repository for future reference

In [1]:
%%html

<a class="github-button" href="https://github.com/aws-samples/responsible_ai_aim325_reduce_hallucinations_for_genai_apps" data-color-scheme="no-preference: light; light: light; dark: dark;" data-icon="octicon-star" data-size="large" data-show-count="true" aria-label="Star Reduce Hallucinations workshop on GitHub">Star</a>
<script async defer src="https://buttons.github.io/buttons.js"></script>

# Environment Setup

In [2]:
#%pip install --upgrade --quiet pip sagemaker boto3 ragas==0.1.7 pydantic==2.6.1 langchain-core==0.1.40 langchain langchain-aws

In [ ]:
#%%capture
!pip3 install -r ../requirements.txt --quiet

In [4]:
!pip list | grep datasets

datasets                                3.1.0


In [5]:
# restart kernel
#from IPython.core.display import HTML
#HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [6]:
import time
import os
import json
import boto3
from time import gmtime, strftime, sleep
import pprint
import random
import zipfile
#from retrying import retry
from rag_setup.create_kb_utils import *
import warnings
warnings.filterwarnings('ignore')
from botocore.config import Config

import numpy as np  
import pandas as pd 
import sagemaker
from botocore.exceptions import ClientError

(sagemaker.__version__,boto3.__version__)

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_id" in BedrockBase has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in BedrockBase has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


('2.227.0', '1.35.15')

## Set constants

In [7]:
# Get some variables you need to interact with SageMaker service
boto_session = boto3.Session()
region = boto_session.region_name
bucket_name = sagemaker.Session().default_bucket()
bucket_prefix = "reduce-hallucinations-in-genai-apps"  
sm_session = sagemaker.Session()
sm_client = boto_session.client("sagemaker")
sm_role = sagemaker.get_execution_role()

initialized = True

print(sm_role)
print(bucket_name)

arn:aws:iam::996757723911:role/cfn-SageMakerExecutionRole-bxMsce3pMtli
sagemaker-us-west-2-996757723911


In [8]:
embedding_model_id="amazon.titan-embed-text-v2:0"
llm_model_id="anthropic.claude-3-sonnet-20240229-v1:0"

In [9]:
# Store some variables to keep the value between the notebooks
%store bucket_name
%store bucket_prefix
%store sm_role
%store region
%store initialized

Stored 'bucket_name' (str)
Stored 'bucket_prefix' (str)
Stored 'sm_role' (str)
Stored 'region' (str)
Stored 'initialized' (bool)


In [10]:
#test if bedrock model access has been enabled 
input_prompt = "Who was the first person to land on the sun?"
test_llm_call(input_prompt)

/home/sagemaker-user/responsible_ai_reduce_hallucinations_for_genai_apps/reduce_llm_hallucinations_labs/lab1/rag_setup/create_kb_utils.py:60: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response = llm(messages)


"No one has ever landed on the sun. The sun is a star with extremely hot temperatures and harsh conditions that make landing on its surface impossible with current technology.\n\nThe sun's surface temperature is around 5,500°C (9,940°F). Its powerful gravitational pull and lack of a solid surface also make landing unfeasible. Any spacecraft would burn up long before reaching the sun's surface due to the intense heat and radiation.\n\nSpace exploration has focused on studying the sun from a safe distance using spacecraft like the Parker Solar Probe, which has flown through the sun's outer atmosphere but not landed. Landing a crew or rover on the sun remains the stuff of science fiction for now. The extreme conditions simply make it impossible to achieve with present technology and materials."

# 1. Chat with Anthropic Claude 3 Sonnet through Bedrock

In [11]:
boto_session = boto3.Session()
region = boto_session.region_name

RETRY_CONFIG = Config(
    retries={
        'max_attempts': 5,            # Maximum number of retry attempts
        'mode': 'adaptive'            # Adaptive mode adjusts based on request limits
    },
    read_timeout=1000,
    connect_timeout=1000
)

bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name=region,
    config=RETRY_CONFIG)

def generate_message_claude(
    query, system_prompt="", max_tokens=1000, 
    model_id='anthropic.claude-3-sonnet-20240229-v1:0',
    temperature=0.9, top_p=0.99, top_k=100
):
    # Prompt with user turn only.
    user_message = {"role": "user", "content": query}
    messages = [user_message]
    body = json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "system": system_prompt,
            "messages": messages,
            "temperature": temperature,
            "top_p": top_p,
            "top_k": top_k
        }
    )

    response = bedrock_runtime.invoke_model(body=body, modelId=model_id)
    response_body = json.loads(response.get('body').read())
    return response_body

In [12]:
query = 'How do Amazon Bedrock Guardrails work?'

response = generate_message_claude(query)
print("User turn only.")
print(json.dumps(response, indent=4))

User turn only.
{
    "id": "msg_bdrk_01SY4qBYRaf7eknVNyeRdHws",
    "type": "message",
    "role": "assistant",
    "model": "claude-3-sonnet-20240229",
    "content": [
        {
            "type": "text",
            "text": "Amazon Bedrock Guardrails is a service provided by AWS that helps organizations govern their AWS environments at scale. It allows organizations to define and enforce rules, known as guardrails, across their AWS accounts and resources.\n\nHere's how Amazon Bedrock Guardrails works:\n\n1. Guardrail Definition: Organizations can define guardrails using AWS CloudFormation templates. These templates specify the desired configuration for AWS resources, services, and account settings. Guardrails can cover various aspects, such as resource naming conventions, security controls, cost optimization, and compliance requirements.\n\n2. Guardrail Deployment: Once the guardrails are defined, they are deployed across the organization's AWS accounts and regions using AWS Organ

## 1.1 Apply System Prompt

In [13]:
query = 'Is it possible to purchase provisioned throughput for Anthropic Claude models on Amazon Bedrock?'
system_prompt = 'You are a helpful AI assistant. You try to answer the user queries to the best of your knowledge. If you are unsure of the answer, do not make up any information.'

response = generate_message_claude(query, system_prompt)
print("User turn only.")
print(json.dumps(response, indent=4))

User turn only.
{
    "id": "msg_bdrk_01BgvsJKdMHV4MXiKQ9D7VVX",
    "type": "message",
    "role": "assistant",
    "model": "claude-3-sonnet-20240229",
    "content": [
        {
            "type": "text",
            "text": "Unfortunately, I do not have any specific information about provisioning throughput or using Anthropic's models, including myself, on Amazon Bedrock. Amazon Bedrock appears to be an internal Amazon service, and I do not have details about its capabilities or offerings related to AI models from Anthropic or other providers. My knowledge is limited to what is publicly available about me and Anthropic's products and services."
        }
    ],
    "stop_reason": "end_turn",
    "stop_sequence": null,
    "usage": {
        "input_tokens": 65,
        "output_tokens": 89
    }
}


In [14]:
query = 'How do Amazon Bedrock Guardrails work?'
system_prompt = 'You are a helpful AI assistant. You try to answer the user queries to the best of your knowledge. If you are unsure of the answer, do not make up any information.'

response = generate_message_claude(query, system_prompt)
print("User turn only.")
print(json.dumps(response, indent=4))

User turn only.
{
    "id": "msg_bdrk_01SWStSGNUMVqkSpwVDAuS5K",
    "type": "message",
    "role": "assistant",
    "model": "claude-3-sonnet-20240229",
    "content": [
        {
            "type": "text",
            "text": "Amazon Bedrock Guardrails is a service provided by AWS that allows you to establish mandatory, organization-wide governance guardrails for your AWS accounts and resources. It helps ensure your workloads conform to your organization's policies before resources are provisioned.\n\nHere's a high-level overview of how Amazon Bedrock Guardrails works:\n\n1. Guardrail Definition: You define guardrails as code using AWS CloudFormation templates or Terraform configurations. These guardrails encode your organization's policies and best practices.\n\n2. Guardrail Deployment: The guardrail definitions are deployed as stacks across your AWS accounts and regions using a centralized deployment mechanism provided by Bedrock Guardrails.\n\n3. Preventive Controls: The deployed

## 1.2 Understanding LLM generation parameters
### 1. Temperature: The amount of randomness injected into the response.

In [15]:
query = 'What is Amazon Bedrock?'
system_prompt = 'You are a helpful AI assistant. You try to answer the user queries to the best of your knowledge. If you are unsure of the answer, do not make up any information.'

response = generate_message_claude(query, system_prompt, temperature=1)
print("User turn only.")
print(json.dumps(response, indent=4))

User turn only.
{
    "id": "msg_bdrk_015s1hUdXGR5RgLrPfzFLqvX",
    "type": "message",
    "role": "assistant",
    "model": "claude-3-sonnet-20240229",
    "content": [
        {
            "type": "text",
            "text": "Amazon Bedrock is a real-time operating system developed by Amazon for use in embedded systems and internet of things (IoT) devices.\n\nSome key points about Amazon Bedrock:\n\n- It is a Linux-based operating system optimized for secure IoT applications and microcontroller-based devices.\n\n- It provides a minimal trusted code base with real-time performance for constrained devices with limited memory and storage.\n\n- It includes built-in support for over-the-air (OTA) updates to allow remote and secure updates of the software on IoT devices.\n\n- It supports C and C++ programming languages.\n\n- Amazon Bedrock aims to provide a secure, real-time foundation for building IoT products across various sectors like industrial, automotive, consumer, etc.\n\n- It wa

In [16]:
query = 'What is Amazon Bedrock?'
system_prompt = 'You are a helpful AI assistant. You try to answer the user queries to the best of your knowledge. If you are unsure of the answer, do not make up any information.'

response = generate_message_claude(query, system_prompt, temperature=0)
print("User turn only.")
print(json.dumps(response, indent=4))

User turn only.
{
    "id": "msg_bdrk_01Xfi93y9pwQBkhE36pJWfW6",
    "type": "message",
    "role": "assistant",
    "model": "claude-3-sonnet-20240229",
    "content": [
        {
            "type": "text",
            "text": "Amazon Bedrock is a real-time operating system developed by Amazon for running applications on resource-constrained devices like microcontrollers and sensors.\n\nSome key points about Amazon Bedrock:\n\n- It is designed to be a secure, real-time operating system for internet of things (IoT) devices and embedded applications.\n\n- It provides a lightweight environment with real-time performance for running multiple software components concurrently.\n\n- It supports C and C++ programming languages.\n\n- It includes built-in security features like memory protection, encrypted communication, secure boot, and code signing.\n\n- It aims to simplify development and deployment of IoT applications across different hardware platforms.\n\n- Bedrock is open source and ava

#### 2. top_p – Use nucleus sampling.

In nucleus sampling, Anthropic Claude computes the cumulative distribution over all the options for each subsequent token in decreasing probability order and cuts it off once it reaches a particular probability specified by top_p. You should alter either temperature or top_p, but not both.

In [17]:
query = 'What is Amazon Bedrock?'
system_prompt = 'You are a helpful AI assistant. You try to answer the user queries to the best of your knowledge. If you are unsure of the answer, do not make up any information.'

response = generate_message_claude(query, system_prompt, temperature=1, top_p=1)
print("User turn only.")
print(json.dumps(response, indent=4))

User turn only.
{
    "id": "msg_bdrk_019nwMrWdiHtJnLuvnsXcgum",
    "type": "message",
    "role": "assistant",
    "model": "claude-3-sonnet-20240229",
    "content": [
        {
            "type": "text",
            "text": "Amazon Bedrock is a real-time operating system (RTOS) developed by Amazon Web Services (AWS) for running lightweight IoT applications on resource-constrained devices.\n\nSome key features of Amazon Bedrock:\n\n1) Small footprint: It is designed to have a minimal memory footprint, making it suitable for microcontrollers and other devices with limited memory and compute resources.\n\n2) Real-time performance: As an RTOS, it provides real-time scheduling and execution capabilities required for time-sensitive IoT applications.\n\n3) Secure: It includes security features like memory protection, cryptographic libraries, and secure boot capabilities.\n\n4) Connected: It supports connectivity to AWS IoT services, allowing devices to easily integrate with the AWS cloud

#### 3. top_k: Only sample from the top K options for each subsequent token.

Use top_k to remove long tail low probability responses.

In [18]:
query = 'What is Amazon Bedrock?'
system_prompt = 'You are a helpful AI assistant. You try to answer the user queries to the best of your knowledge. If you are unsure of the answer, do not make up any information.'

response = generate_message_claude(query, system_prompt, temperature=0, top_p=1, top_k=100)
print("User turn only.")
print(json.dumps(response, indent=4))

User turn only.
{
    "id": "msg_bdrk_01SkA6egzjn9mcCuSTZVAgZB",
    "type": "message",
    "role": "assistant",
    "model": "claude-3-sonnet-20240229",
    "content": [
        {
            "type": "text",
            "text": "Amazon Bedrock is a real-time operating system developed by Amazon for running applications on resource-constrained devices like microcontrollers and sensors.\n\nSome key points about Amazon Bedrock:\n\n- It is designed to be a secure, real-time operating system for internet of things (IoT) devices and embedded applications.\n\n- It provides a lightweight environment with real-time performance for running multiple software components concurrently.\n\n- It supports C and C++ programming languages.\n\n- It includes built-in security features like memory protection, encrypted communication, secure boot, and code signing.\n\n- It aims to simplify development and deployment of IoT applications across different hardware platforms.\n\n- Bedrock is open source and ava

# Retrieval Augmented Generation
We are using the Retrieval Augmented Generation (RAG) technique with Amazon Bedrock. A RAG implementation consists of two parts:

    1. A data pipeline that ingests that from documents (typically stored in Amazon S3) into a Knowledge Base i.e. a vector database such as Amazon OpenSearch Service Serverless (AOSS) so that it is available for lookup when a question is received.

The data pipeline represents an undifferentiated heavy lifting and can be implemented using Amazon Bedrock Knowledge Bases. We can now connect an S3 bucket to a vector database such as AOSS and have a Bedrock Knowledge Bases read the objects (html, pdf, text etc.), chunk them, and then convert these chunks into embeddings using Amazon Titan Embeddings model and then store these embeddings in AOSS. All of this without having to build, deploy, and manage the data pipeline.

<center><img src="images/fully_managed_ingestion.png" alt="This image shows how Aazon Bedrock Knowledge Bases ingests objects in a S3 bucket into the Knowledge Base for use in a RAG set up. The objects are chunks, embedded and then stored in a vector index." height="700" width="700" style="background-color:white; padding:1em;" /></center> <br/>
    

    2. An application that receives a question from the user, looks up the knowledge base for relevant pieces of information (context) and then creates a prompt that includes the question and the context and provides it to an LLM for generating a response.






Once the data is available in the Bedrock knowledge base, then user questions can be answered using the following system design:

<center><img src="images/retrieveAndGenerate.png" alt="This image shows the retrieval augmented generation (RAG) system design setup with knowledge bases, S3, and AOSS. Knowledge corpus is ingested into a vector database using Amazon Bedrock Knowledge Base Agent and then RAG approach is used to work question answering. The question is converted into embeddings followed by semantic similarity search to get similar documents. With the user prompt being augmented with the RAG search response, the LLM is invoked to get the final raw response for the user." height="700" width="700" style="background-color:white; padding:1em;" /></center> <br/>


# Data
Let's use the publicly available [Bedrock user guide](https://docs.aws.amazon.com/pdfs/bedrock/latest/userguide/bedrock-ug.pdf) to inform the model

In [19]:
!wget -P data/ -N https://docs.aws.amazon.com/pdfs/bedrock/latest/userguide/bedrock-ug.pdf --no-check-certificate

--2024-11-13 20:31:22--  https://docs.aws.amazon.com/pdfs/bedrock/latest/userguide/bedrock-ug.pdf
Resolving docs.aws.amazon.com (docs.aws.amazon.com)... 18.238.238.32, 18.238.238.78, 18.238.238.98, ...
Connecting to docs.aws.amazon.com (docs.aws.amazon.com)|18.238.238.32|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 13669946 (13M) [application/pdf]
Saving to: ‘data/bedrock-ug.pdf’

bedrock-ug.pdf      100%[===================>]  13.04M  --.-KB/s    in 0.1s    

2024-11-13 20:31:23 (112 MB/s) - ‘data/bedrock-ug.pdf’ saved [13669946/13669946]



In [20]:
# Upload data to S3
dataset_file_local_path = 'data/bedrock-ug.pdf'
input_s3_url = sagemaker.Session().upload_data(
    path=dataset_file_local_path,
    bucket=bucket_name
)
print(f"Upload the dataset to {input_s3_url}")

%store input_s3_url

Upload the dataset to s3://sagemaker-us-west-2-996757723911/data/bedrock-ug.pdf
Stored 'input_s3_url' (str)


# Steps

1. Create Amazon Bedrock Knowledge Base execution role with necessary policies for accessing data from S3 and writing embeddings into OSS.
2. Create an empty OpenSearch serverless index.
3. Create Amazon Bedrock knowledge base
4. Create a data source within knowledge base which will connect to Amazon S3
5. Start an ingestion job using KB APIs which will read data from s3, chunk it, convert chunks into embeddings using Amazon Titan Embeddings model and then store these embeddings in AOSS. 

In [21]:
!export PYTHONPATH='./lab1/'
#import sys
#sys.path.insert(0,'./lab1/')

In [22]:
kb_db_file_uri='data'

# if a kb already exists we can use the same, else the infra setup code will create one by itself using the bedrock user guide.
use_existing_kb = False
existing_kb_id = None

In [23]:
%load_ext autoreload
%autoreload 2
from rag_setup.create_kb_utils import *

In [24]:
%%time

# For new KB it takes around ~6 minutes for this setup to complete on a t2.medium instance.
infra_response = setup_knowledge_base(bucket_name, kb_db_file_uri, use_existing_kb, existing_kb_id)
infra_response

agent_bedrock_policy :: None
agent_s3_schema_policy :: None
kb_aws_bedrock_policy :: None
kb_db_s3_policy :: None
Creating collection...

Collection successfully created:

Creating index:
Knowledge base status -> is it READY ? :: ACTIVE
knowledge_base_db_id :: DSALKIXRIV
CPU times: user 275 ms, sys: 33.1 ms, total: 308 ms
Wall time: 5min 49s


{'prefix_infra': 'l2ad2e45',
 'bucket_name': 'sagemaker-us-west-2-996757723911',
 'knowledge_base_db_id': 'DSALKIXRIV',
 'agent_bedrock_policy': None,
 'agent_s3_schema_policy': None,
 'kb_db_collection_name': 'l25929-kbdb-996757723911',
 'agent_kb_schema_policy': None,
 'kb_db_aoss_policy': None,
 'kb_db_s3_policy': None,
 'kb_db_role_name': 'AmazonBedrockExecutionRoleForAgentsAIAssistant05',
 'kb_db_opensearch_collection_response': {'createCollectionDetail': {'arn': 'arn:aws:aoss:us-west-2:996757723911:collection/zrswlgcemuvny02ltvph',
   'createdDate': 1731529886011,
   'description': 'OpenSearch collection for Amazon Bedrock Latest User guide Knowledge Base',
   'id': 'zrswlgcemuvny02ltvph',
   'kmsKeyArn': 'auto',
   'lastModifiedDate': 1731529886011,
   'name': 'l25929-kbdb-996757723911',
   'standbyReplicas': 'DISABLED',
   'status': 'CREATING',
   'type': 'VECTORSEARCH'},
  'ResponseMetadata': {'RequestId': 'b33b1a24-31b0-450f-b481-c1bd338f9ebf',
   'HTTPStatusCode': 200,
   'H

In [25]:
kb_id = infra_response['knowledge_base_db_id']
random_id = infra_response['prefix_infra']
# keep the kb_id for invocation later in the invoke request
%store kb_id
%store bucket_name

Stored 'kb_id' (str)
Stored 'bucket_name' (str)


In [26]:
kb_id

'DSALKIXRIV'

In [27]:
# allow time for KB to be ready
time.sleep(180)

# Chat with the model using the knowledge base by providing the generated KB_ID
### Using RetrieveAndGenerate API
Behind the scenes, RetrieveAndGenerate API converts queries into embeddings, searches the knowledge base, and then augments the foundation model prompt with the search results as context information and returns the FM-generated response to the question. For multi-turn conversations, Knowledge Bases manage short-term memory of the conversation to provide more contextual results.The output of the RetrieveAndGenerate API includes the generated response, source attribution as well as the retrieved text chunks.

In [28]:
pp = pprint.PrettyPrinter(indent=2)

In [29]:
kb_id

'DSALKIXRIV'

In [30]:
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region_name=region)


def ask_bedrock_llm_with_knowledge_base(query,
                                        kb_id=kb_id,
                                        model_arn=llm_model_id,
                                        ) -> str:
    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={
            'text': query
        },
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': kb_id,
                'modelArn': model_arn
            }
        },
    )

    return response

In [31]:
query = "What is Amazon Bedrock?"

response = ask_bedrock_llm_with_knowledge_base(query, kb_id)
generated_text = response['output']['text']
citations = response["citations"]
contexts = []
for citation in citations:
    retrievedReferences = citation["retrievedReferences"]
    for reference in retrievedReferences:
        contexts.append(reference["content"]["text"])
print(f"---------- Generated using Anthropic Claude 3 Sonnet:")
pp.pprint(generated_text )
print(f'---------- The citations for the response:')
pp.pprint(contexts)
print(kb_id)

---------- Generated using Anthropic Claude 3 Sonnet:
('Amazon Bedrock is a fully managed service that provides access to '
 'high-performing foundation models (FMs) from leading AI companies and Amazon '
 'through a unified API. It allows you to experiment with and evaluate '
 'different foundation models, customize them with your own data using '
 'techniques like fine-tuning and Retrieval Augmented Generation (RAG), and '
 'build agents that can execute tasks using your systems and data sources. '
 "With Amazon Bedrock's serverless experience, you can get started quickly, "
 'customize foundation models with your data, and easily integrate and deploy '
 'them into your applications using AWS tools without managing any '
 'infrastructure.')
---------- The citations for the response:
[ '............ 2009     xvAmazon Bedrock User Guide     What is Amazon '
  'Bedrock?     Amazon Bedrock is a fully managed service that makes '
  'high-performing foundation models (FMs) from leading AI 

In [32]:
query = "Is it possible to purchase provisioned throughput for Anthropic Claude Sonnet on Amazon Bedrock?"

response = ask_bedrock_llm_with_knowledge_base(query, kb_id)
generated_text = response['output']['text']
citations = response["citations"]
contexts = []
for citation in citations:
    retrievedReferences = citation["retrievedReferences"]
    for reference in retrievedReferences:
        contexts.append(reference["content"]["text"])
print(f"---------- Generated using Anthropic Claude 3 Sonnet:")
pp.pprint(generated_text )
print(f'---------- The citations for the response:')
pp.pprint(contexts)
print()

---------- Generated using Anthropic Claude 3 Sonnet:
('Yes, it is possible to purchase provisioned throughput for Anthropic Claude '
 'Sonnet models on Amazon Bedrock. Specifically, you can purchase provisioned '
 'throughput for the following Anthropic Claude Sonnet models:\n'
 '\n'
 '- Anthropic Claude 3 Sonnet 28K\n'
 '- Anthropic Claude 3 Sonnet 200K\n'
 '- Anthropic Claude 3.5 Sonnet 18K (only available in US West (Oregon) '
 'region)\n'
 '- Anthropic Claude 3.5 Sonnet 51K (only available in US West (Oregon) '
 'region)\n'
 '- Anthropic Claude 3.5 Sonnet 200K (only available in US West (Oregon) '
 'region)')
---------- The citations for the response:
[ 'Virginia)     US West (Oregon)     Asia Pacific (Mumbai)     Asia Pacific '
  '(Sydney)     Canada (Central)     Europe (London)     Europe (Paris)     '
  'Europe (Ireland)     South America (São Paulo)     AWS GovCloud (US-West) '
  '(only for custom models with no commitment)     If you purchase Provisioned '
  'Throughput thro

# Contextual Grounding with Amazon Bedrock Guardrails

In [33]:
# Create guardrail
bedrock_client = boto3.client('bedrock')
guardrail_name = f"bedrock-rag-grounding-guardrail-{random_id}"
print(guardrail_name)
guardrail_response = bedrock_client.create_guardrail(
    name=guardrail_name,
    description='Guardrail for ensuring relevance and grounding of model responses in RAG powered chatbot',
    contextualGroundingPolicyConfig={
        'filtersConfig': [
            {
                'type': 'GROUNDING',
                'threshold': 0.5
            },
            {
                'type': 'RELEVANCE',
                'threshold': 0.5
            },
        ]
    },
    blockedInputMessaging='Can you please rephrase your question?',
    blockedOutputsMessaging='Sorry, I am not able to find the correct answer to your query - Can you try reframing your query to be more specific'
)

bedrock-rag-grounding-guardrail-l2ad2e45


In [34]:
guardrailId = guardrail_response['guardrailId']
guardrail_response

{'ResponseMetadata': {'RequestId': '8d13ae15-11c1-4baa-9cf5-850bb9bf54d5',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Wed, 13 Nov 2024 20:40:26 GMT',
   'content-type': 'application/json',
   'content-length': '172',
   'connection': 'keep-alive',
   'x-amzn-requestid': '8d13ae15-11c1-4baa-9cf5-850bb9bf54d5'},
  'RetryAttempts': 0},
 'guardrailId': '5qsclikqvcxn',
 'guardrailArn': 'arn:aws:bedrock:us-west-2:996757723911:guardrail/5qsclikqvcxn',
 'version': 'DRAFT',
 'createdAt': datetime.datetime(2024, 11, 13, 20, 40, 26, 553590, tzinfo=tzlocal())}

In [35]:
guardrail_version = bedrock_client.create_guardrail_version(
    guardrailIdentifier=guardrail_response['guardrailId'],
    description='Working version of RAG app guardrail with higher thresholds for contextual grounding'
)
print(guardrail_version)
guardrailVersion = guardrail_response['version']
print(guardrailId)
%store guardrailId

{'ResponseMetadata': {'RequestId': '501376b1-218d-4d0b-9aff-a57a9a42444b', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Wed, 13 Nov 2024 20:40:27 GMT', 'content-type': 'application/json', 'content-length': '44', 'connection': 'keep-alive', 'x-amzn-requestid': '501376b1-218d-4d0b-9aff-a57a9a42444b'}, 'RetryAttempts': 0}, 'guardrailId': '5qsclikqvcxn', 'version': '1'}
5qsclikqvcxn
Stored 'guardrailId' (str)


In [36]:
# Retrieve and Generate using Guardrail

bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region_name=region)


def retrieve_and_generate_with_guardrail(
    query,
    kb_id,
    model_arn=llm_model_id,
    session_id=None
):

    prompt_template = 'You are a helpful AI assistant to help users understand documented risks in various projects. \
    Answer the user query based on the context retrieved. If you dont know the answer, dont make up anything. \
    Only answer based on what you know from the provided context. You can ask the user for clarifying questions if anything is unclear\
    But generate an answer only when you are confident about it and based on the provided context.\
    User Query: $query$\
    Context: $search_results$'

    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={
            'text': query
        },
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'generationConfiguration': {
                    'guardrailConfiguration': {
                        'guardrailId': guardrailId,
                        'guardrailVersion': guardrailVersion
                    },
                    'inferenceConfig': {
                        'textInferenceConfig': {
                            'temperature': 0.7,
                            'topP': 0.25
                        }
                    },
                    'promptTemplate': {
                        'textPromptTemplate': prompt_template
                    }
                },
                'knowledgeBaseId': kb_id,
                'modelArn': model_arn,
                'retrievalConfiguration': {
                    'vectorSearchConfiguration': {
                        'overrideSearchType': 'SEMANTIC'
                    }
                }
            }
        }
    )
    return response

In [37]:
# Knowledge BAse ID

query = 'What is Amazon Bedrock?'
#query = "Is it possible to purchase provisioned throughput for Anthropic Claude Sonnet on Amazon Bedrock?"

model_response = retrieve_and_generate_with_guardrail(query, kb_id)

print(model_response)

{'ResponseMetadata': {'RequestId': 'c527ebad-1b09-45e7-9ed3-2c6fb0e059ae', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 13 Nov 2024 20:40:36 GMT', 'content-type': 'application/json', 'content-length': '1383', 'connection': 'keep-alive', 'x-amzn-requestid': 'c527ebad-1b09-45e7-9ed3-2c6fb0e059ae'}, 'RetryAttempts': 0}, 'citations': [], 'guardrailAction': 'NONE', 'output': {'text': 'According to the context provided, Amazon Bedrock is a fully managed service from AWS that provides access to high-performing foundation models (FMs) from leading AI companies and Amazon through a unified API.\n\nSome key points about Amazon Bedrock:\n\n- It allows you to choose from a wide range of foundation models to find the best one for your use case.\n- It offers capabilities to build generative AI applications with security, privacy, and responsible AI principles.\n- You can experiment with and evaluate different foundation models, privately customize them with your own data using techniques lik

# Evaluating RAG with RAGAS

In [38]:
import boto3
import pprint
from botocore.client import Config
from langchain.llms.bedrock import Bedrock
from langchain_community.chat_models.bedrock import BedrockChat
from langchain.embeddings import BedrockEmbeddings
from langchain.retrievers.bedrock import AmazonKnowledgeBasesRetriever
from langchain.chains import RetrievalQA

pp = pprint.PrettyPrinter(indent=2)

bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime')
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config
                              )

llm_for_text_generation = BedrockChat(model_id=llm_model_id, client=bedrock_client)

llm_for_evaluation = BedrockChat(model_id=llm_model_id, client=bedrock_client)

bedrock_embeddings = BedrockEmbeddings(model_id=embedding_model_id,client=bedrock_client)

/tmp/ipykernel_3982/422286211.py:22: LangChainDeprecationWarning: The class `BedrockEmbeddings` was deprecated in LangChain 0.2.11 and will be removed in 1.0. An updated version of the class exists in the langchain-aws package and should be used instead. To use it run `pip install -U langchain-aws` and import as `from langchain_aws import BedrockEmbeddings`.
  bedrock_embeddings = BedrockEmbeddings(model_id=embedding_model_id,client=bedrock_client)


In [39]:
import pandas as pd

test = pd.read_csv('data/bedrock-user-guide-test.csv')
test = test.dropna()
test.style.set_properties(**{'text-align': 'left', 'border': '1px solid black'})
test.to_string(justify='left', index=False)
with pd.option_context("display.max_colwidth", None):
    pretty_print(test)

,Question/prompt,Correct answer
0,Are all models accessible on Amazon Bedrock by default?,"Access to Amazon Bedrock foundation models isn't granted by default. You can request access, or modify access, to foundation models only by using the Amazon Bedrock console. First, make sure the IAM role that you use has sufficent IAM permissions to manage access to foundation models. Then, add or remove access to a model by following the instructions at Add or remove access to Amazon Bedrock foundation models."
1,What is the Model ID of Amazon Titan Text Premier,amazon.titan-text-premier-v1:0
2,With which Anthropic Claude models can I use the Text Completions API?,"Anthropic Claude Instant v1.2, Anthropic Claude v2, Anthropic Claude v2.1"
3,What policies can I configure in Amazon Bedrock guardrails?,"You can configure the following policies in a guardrail to avoid undesirable and harmful content and remove sensitive information for privacy protection. Content filters – Adjust filter strengths to block input prompts or model responses containing harmful content.Denied topics – Define a set of topics that are undesirable in the context of your application. These topics will be blocked if detected in user queries or model responses.Word filters – Configure filters to block undesirable words, phrases, and profanity. Such words can include offensive terms, competitor names etc.Sensitive information filters – Block or mask sensitive information such as personally identifiable information (PII) or custom regex in user inputs and model responses.Contextual grounding check – Detect and filter hallucinations in model responses based on grounding in a source and relevance to the user query."
4,Which built in datasets are available on Amazon Bedrock for model evaluation of text generation?,"The following built-in datasets contain prompts that are well-suited for use in general text generation tasks. Bias in Open-ended Language Generation Dataset (BOLD)The Bias in Open-ended Language Generation Dataset (BOLD) is a dataset that evaluates fairness in general text generation, focusing on five domains: profession, gender, race, religious ideologies, and political ideologies. It contains 23,679 different text generation prompts.RealToxicityPromptsRealToxicityPrompts is a dataset that evaluates toxicity. It attempts to get the model to generate racist, sexist, or otherwise toxic language. This dataset contains 100,000 different text generation prompts.T-Rex : A Large Scale Alignment of Natural Language with Knowledge Base Triples (TREX)TREX is dataset consisting of Knowledge Base Triples (KBTs) extracted from Wikipedia. KBTs are a type of data structure used in natural language processing (NLP) and knowledge representation. They consist of a subject, predicate, and object, where the subject and object are linked by a relation. An example of a Knowledge Base Triple (KBT) is ""George Washington was the president of the United States"". The subject is ""George Washington"", the predicate is ""was the president of"", and the object is ""the United States"".WikiText2WikiText2 is a HuggingFace dataset that contains prompts used in general text generation."


In [40]:
from datasets import Dataset

questions = test['Question/prompt'].tolist()
ground_truths = [[gt] for gt in test['Correct answer'].tolist()]

answers = []
contexts = []

for query in questions:
    response = ask_bedrock_llm_with_knowledge_base(query, kb_id)
    generatedResult = response['output']['text']
    answers.append(generatedResult)
    contexts.append([doc['content']['text'] for doc in response['citations'][0]['retrievedReferences']])

# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

In [41]:
#!pip install datasets -U

In [1]:
!pip list | grep datasets

datasets                                3.1.0


In [ ]:
#%%capture
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    context_entity_recall,
    answer_similarity,
    answer_correctness
)

#specify the metrics here, kept one for now, we can add more.
metrics = [
        answer_relevancy
    ]

result = evaluate(
    dataset = dataset, 
    metrics=metrics,
    llm=llm_for_evaluation,
    embeddings=bedrock_embeddings,
    raise_exceptions=False
)

ragas_df = result.to_pandas()

In [43]:
ragas_df.style.set_properties(**{'text-align': 'left', 'border': '1px solid black'})
ragas_df.to_string(justify='left', index=False)
with pd.option_context("display.max_colwidth", None):
    pretty_print(ragas_df)

### <a >Challenge Exercise :: Try it Yourself! </a>


<div style="border: 4px solid coral; text-align: left; margin: auto;">
    <br>
    <p style="text-align: center; margin: auto;"><b>Try the following exercises on this lab and note the observations.</b></p>
<p style=" text-align: left; margin: auto;">
<ol>
    <li>Test the RAG based LLM with more questions about Amazon Bedrock. </li>
<li>Look the the citations or retrieved references and see if the answer generated by the RAG chatbot aligns with these retrieved contexts. What response do you get when the retrieved context comes up empty? </li>
<li>Apply system prompts to RAG as well as amazon Bedrock Guardrails and test which is more consistent in blocking responses when the model response is hallucinated </li>
<li>Run the tutorial for RAG Checker and compare the difference with RAGAS evaluation framework: https://github.com/amazon-science/RAGChecker/blob/main/tutorial/ragchecker_tutorial_en.md </li>
</ol>
<br>
</p>
</div>


## Conclusion
We now have an understanding of parameters which influence hallucinations in Large Language Models. We learnt how to set up Retrieval Augmented Generation to provide a context to the model while answering.
We used Contextual grounding in Amazon Bedrock Guardrials to intervene when hallucinations are detected.
Finally we looked into the metrics of RAGAS and how to use them to measure hallucinations in your RAG powered chatbot.

In the next lab, we will:
1. Build a custom hallucination detector
2. Use Amazon Bedrock Agents to intervene when hallucinations are detected
3. Call a human for support when the LLM hallucinates
